# Stage 01: Setup 
## Step 01: Initialize

This step initializes the workflow and validates the environment.

**Purpose:**
- Verify database connectivity
- Validate cycle configuration
- Set up working directories
- Initialize logging

In [ ]:
from system.context import WorkContext
from system.step import Step
from system import database, ux, config as cfg
# Create work context - automatically detects cycle/stage/step from notebook path
context = WorkContext()
print(f"{context}")

Context detected: Q4_2025_Demo-Analysis → Stage 1 → Step 1
Database entries ready (step_id=1)
Q4_2025_Demo-Analysis → Stage 1: Setup → Step 1: Initialize


In [ ]:
# Create step tracker
step = Step(context)

# Check if already execute
if step.executed:
    ux.warning(step.status_message)
    raise Exception(step.status_message)
ux.success("Safe to proceed")

Exception: Step already run, in status: running

In [4]:
step.start(force=True)

Starting Step Run #2
Q4_2025_Demo-Analysis → Stage 1: Setup → Step 1: Initialize
Run ID: 2
(i) Step execution started


In [ ]:
print("\n1. Testing database connection...")
if database.test_connection():
    ux.success("Database connected")
    step.log("Database connection verified")
else:
    raise Exception("Database connection failed")

# 2. Verify active cycle
print("\n2. Verifying active cycle...")
active = database.get_active_cycle()
if active:
    ux.success(f"Active cycle: {active['cycle_name']}")
    step.log(f"Active cycle verified: {active['cycle_name']}")
else:
    raise Exception("No active cycle found")

# 3. Set up working directories
print("\n3. Setting up /files directory...")
from pathlib import Path

cycle_dir = Path(f"/home/jovyan/workspace/workflows/Active_{active['cycle_name']}")
files_dir = cycle_dir / "files"    
logs_dir = cycle_dir / "logs"    
files_dir = cycle_dir / "files"    
logs_dir = cycle_dir / "files"    

ux.success(f"Files directory: {files_dir}")
ux.success(f"Logs directory: {logs_dir}")
step.log("Directory created")

# Save checkpoint
step.checkpoint({
    'cycle_dir': str(cycle_dir),
    'files_dir': str(files_dir),
    'logs_dir': str(files_dir)
})

import logging
from datetime import datetime

log_file = logs_dir / f"initialization_{datetime.now().strftime('%Y%m%d_%H%M%S')}.log"
logging.basicConfig(
    filename=str(log_file),
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)
logging.info("Workflow initialized")

ux.success(f"Log file: {log_file}")
step.log(f"Logging initialized: {log_file}")

# Prepare output data
output = {
    'cycle_name': active['cycle_name'],
    'cycle_id': active['id'],
    'files_dir': str(files_dir),
    'logs_dir': str(logs_dir),
    'log_file': str(log_file),
    'initialization_complete': True
}

# Mark step as completed
step.complete(output)

ux.header("INITIALIZATION COMPLETE")


1. Testing database connection...


(i) Database connection verified

2. Verifying active cycle...


(i) Active cycle verified: Q4_2025_Demo-Analysis

3. Setting up /files directory...


(i) Directory created
(i) Checkpoint saved: ['cycle_dir', 'files_dir', 'logs_dir']


(i) Logging initialized: /home/jovyan/workspace/workflows/Active_Q4_2025_Demo-Analysis/files/initialization_20251010_174613.log

STEP COMPLETED
   Run #2 completed in 4.4 seconds


In [ ]:
ux._header("Step Status")

# Get step history
from system import cycle

history = cycle.get_step_history(
    context.cycle_name,
    context.stage_num,
    context.step_num
)

if not history.empty:
    # Format duration
    history['duration'] = history['duration_seconds'].apply(ux.format_duration)
    
    # Display recent runs
    display_df = history[[
        'run_number', 'status', 'started_ts', 'completed_ts', 'duration'
    ]].head(5)
    
    ux.dataframe(display_df, title="Step Run History")
else:
    ux.info("No run history available")

# Show last output
last_output = step.get_last_output()
if last_output:
    ux.subheader("Last Output")
    ux.display_json(last_output, indent=2)

run_number,status,started_ts,completed_ts,duration
2,completed,2025-10-10 17:46:09.218716+00:00,2025-10-10 17:46:13.581283+00:00,4.4s
1,running,2025-10-10 17:45:36.772930+00:00,NaT,nanh
